这个文档是用gluon来训练简单的线性回归模型

In [5]:
from mxnet import autograd, nd

# 随机生成了features+labels

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

print(features)
print(labels)


[[-0.5932887   0.36333826]
 [ 0.5925345  -0.7567092 ]
 [ 0.13926521 -0.20225924]
 ...
 [-0.91519624 -1.2238808 ]
 [-1.6869311  -0.46909225]
 [-0.06433223 -0.06503782]]
<NDArray 1000x2 @cpu(0)>

[ 1.76260602e+00  7.95308113e+00  5.16838551e+00  1.41829996e+01
  6.54783535e+00  3.85844779e+00 -3.70523185e-01  3.97014308e+00
  2.30167651e+00  7.80768299e+00  3.02818298e+00  8.77193260e+00
  7.39109945e+00 -3.97118926e-03  1.02969766e+00  1.33577573e+00
 -1.88204110e-01  2.33395433e+00  1.84126556e+00  6.64658880e+00
  3.65333080e+00  1.42833257e+00  4.32003593e+00  1.02134457e+01
  7.15514517e+00  5.96410275e+00  1.66464174e+00  2.58200812e+00
  6.03774357e+00  1.20682523e-01  5.75148869e+00 -2.40808412e-01
  8.16683960e+00  3.01467347e+00  3.28731632e+00 -4.45818990e-01
  2.15851879e+00  7.74541235e+00  1.07075272e+01  3.17596889e+00
  7.39427984e-01  9.14788723e+00  1.13682306e+00  2.59334517e+00
  2.21794558e+00  1.19434910e+01  6.20424271e+00  4.75233936e+00
  5.70381021e+00  8.72975

In [9]:
from mxnet.gluon import data as gdata

# 制作数据集
batch_size = 10
dataset = gdata.ArrayDataset(features, labels)
print(len(dataset))
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)
print(len(data_iter))

1000
100


In [10]:
for X, y in data_iter:
    print(X, y)
    break


[[ 0.37585694  0.31230175]
 [-0.24758646 -1.3603933 ]
 [ 1.3489803  -3.0220435 ]
 [ 0.03153127  0.18016408]
 [-0.87090194 -0.8708397 ]
 [-0.5883604  -1.5196558 ]
 [ 1.0006329   0.7193293 ]
 [-0.41137263  0.29792118]
 [-0.2876293   1.0307547 ]
 [ 1.3829747  -0.8095842 ]]
<NDArray 10x2 @cpu(0)> 
[ 3.8703396   8.334698   17.175234    3.6523302   5.431855    8.195428
  3.7503488   2.3587744   0.12068252  9.716179  ]
<NDArray 10 @cpu(0)>


In [24]:
from mxnet.gluon import nn
# 定义模型
net = nn.Sequential()
net.add(nn.Dense(1))  
print(len(net))

In [25]:
from mxnet import init
# 初始化模型参数
net.initialize(init.Normal(sigma=0.01))

In [26]:
from mxnet.gluon import loss as gloss
# 定义损失函数
loss = gloss.L2Loss()

In [27]:
from mxnet import gluon
# 定义优化方法
trainer = gluon.Trainer(net.collect_params(),
                        'sgd',
                        {'learning_rate':0.03})

In [28]:
num_epochs = 5
# 开始训练
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' %(epoch, l.mean().asnumpy()))

epoch 1, loss: 0.040422
epoch 2, loss: 0.000150
epoch 3, loss: 0.000050
epoch 4, loss: 0.000050
epoch 5, loss: 0.000050


In [33]:
dense = net[0]
# 我们可以比较一下训练得到的模型参数，和我们当初生成数据集用的参数是否接近？
true_w, dense.weight.data()

(4.2, 
 [4.1999664]
 <NDArray 1 @cpu(0)>)

In [34]:
true_b, dense.bias.data()

(4.2, 
 [4.1999664]
 <NDArray 1 @cpu(0)>)